<a href="https://colab.research.google.com/github/alex-jk/datakit-smallholder-farmers-fall-2025/blob/main/alex_j_farmers_data_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone the repo**

In [1]:
import os
import pandas as pd
from google.colab import drive

repo_dir = "datakit-smallholder-farmers-fall-2025"

if not os.path.isdir(repo_dir):
    !git clone https://github.com/alex-jk/datakit-smallholder-farmers-fall-2025.git

%cd {repo_dir}
%cd 'Prep Challenge- Translation'

/content/datakit-smallholder-farmers-fall-2025
/content/datakit-smallholder-farmers-fall-2025/Prep Challenge- Translation


**Load the farmers survey dataset**

In [2]:
drive.mount('/content/drive')

export_input_to_parquet = False

!pip install duckdb -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import duckdb

csv_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.csv"
parquet_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.parquet"

con = duckdb.connect()

In [4]:
if export_input_to_parquet:

  con.execute(f"""
      COPY (
          SELECT *
          FROM read_csv_auto('{csv_path}', sample_size=-1)
      )
      TO '{parquet_path}'
      (FORMAT PARQUET);
  """)

**Connect to the DB**

In [5]:
parquet_path = "/content/drive/MyDrive/DataKind Farmers Project/farmers_survey_dataset.parquet"

con.execute(f"""
    CREATE OR REPLACE VIEW farmers AS
    SELECT *
    FROM read_parquet('{parquet_path}')
""")

In [10]:
# Row count
row_count_df = con.execute("SELECT COUNT(*) AS n_rows FROM farmers").df()
n_rows = int(row_count_df.loc[0, "n_rows"])

# Column info
cols_info = con.execute("PRAGMA table_info('farmers')").df()
n_cols = cols_info.shape[0]
col_names = cols_info["name"].tolist()

print(f"DF row count: {n_rows}")
print(f"DF column count: {n_cols}")
print("Column names:")
print(col_names)

# Peek at data
con.execute("SELECT * FROM farmers LIMIT 5").df()

DF row count: 20304843
DF column count: 24
Column names:
['question_id', 'question_user_id', 'question_language', 'question_content', 'question_topic', 'question_sent', 'response_id', 'response_user_id', 'response_language', 'response_content', 'response_topic', 'response_sent', 'question_user_type', 'question_user_status', 'question_user_country_code', 'question_user_gender', 'question_user_dob', 'question_user_created_at', 'response_user_type', 'response_user_status', 'response_user_country_code', 'response_user_gender', 'response_user_dob', 'response_user_created_at']


,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,...,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,None,2017-11-22 12:25:03+00:00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',...,ug,None,NaT,2017-11-18 13:09:11+00:00,farmer,live,ug,None,NaT,2017-05-09 09:19:33+00:00
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,None,2017-11-22 12:25:05+00:00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,...,ug,None,NaT,2017-11-20 11:55:48+00:00,farmer,zombie,ug,None,NaT,2017-11-22 10:13:03+00:00
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00:00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,...,ug,None,NaT,2017-08-22 14:51:07+00:00,farmer,zombie,ug,None,NaT,2017-08-12 09:30:33+00:00
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00:00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,...,ug,None,NaT,2017-08-22 14:51:07+00:00,farmer,zombie,ug,None,NaT,2017-08-12 09:30:33+00:00
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00:00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,...,ug,None,NaT,2017-08-22 14:51:07+00:00,farmer,zombie,ug,None,NaT,2017-08-12 09:30:33+00:00


**Check unique question counts**

In [13]:
questions_counts = con.execute("""
    SELECT
        question_id,
        question_topic,
        question_content,
        question_language,
        COUNT(*) AS n_responses
    FROM farmers
    GROUP BY
        question_id,
        question_topic,
        question_content,
        question_language
    ORDER BY
        n_responses DESC
""").df()

questions_counts

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,question_id,question_topic,question_content,question_language,n_responses
0,56318162,bean,Q Which hybrid of beans are resistant at this ...,eng,2984
1,58810648,maize,Q good night ..nataka kuenda kw fam kuchunga m...,swa,2552
2,59076944,tomato,Q If I Tream Tomatoes After harvest can they s...,eng,2112
3,10326500,tomato,E ekibuzo enyanya zange nazifuyide egu...,lug,2043
4,42820499,cattle,QWat are de signs of apregnant cow.,eng,2033
...,...,...,...,...,...
6627404,59256225,plant,Q which crop should we plant in this very litt...,eng,1
6627405,59258899,maize,"Naomba,mbegu ya mahindi..nmeishiwa kabisa na s...",swa,1
6627406,59259045,cabbage,I want to grow cabbage someone to give me the ...,eng,1
6627407,59260982,bird,Q how can i permanently control birds destroyi...,eng,1


**Check if each question ID corresponds to a unique question**

In [15]:
# All question_ids where there is more than 1 distinct question_content
qid_conflicts = con.execute("""
    SELECT
        question_id,
        COUNT(DISTINCT question_content) AS n_question_contents
    FROM farmers
    GROUP BY question_id
    HAVING COUNT(DISTINCT question_content) > 1
    ORDER BY n_question_contents DESC
""").df()

qid_conflicts

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,question_id,n_question_contents


Check english questions and responses

In [22]:
pd.set_option("display.max_colwidth", None)

qid_counts_by_lang = con.execute("""
    SELECT
        question_language,
        COUNT(DISTINCT question_id) AS n_unique_question_ids
    FROM farmers
    GROUP BY question_language
    ORDER BY question_language
""").df()

qid_counts_by_lang["n_unique_question_ids_fmt"] = (
    qid_counts_by_lang["n_unique_question_ids"].astype(int).map("{:,}".format)
)

display(qid_counts_by_lang)

select_lang = "eng"  # or "swa", "lug", ...

sample_lang = con.execute("""
    SELECT
        question_id,
        question_topic,
        question_content,
        response_content,
        question_language
    FROM farmers
    WHERE question_language = ?
      AND question_content IS NOT NULL
      AND response_content IS NOT NULL
    ORDER BY random()
    LIMIT 20
""", [select_lang]).df()

print(f"\n-------Questions Sample {select_lang}------\n")
display(sample_lang)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,question_language,n_unique_question_ids,n_unique_question_ids_fmt
0,eng,2940542,"2,940,542"
1,lug,265958,"265,958"
2,nyn,433345,"433,345"
3,swa,2225974,"2,225,974"


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


-------Questions Sample eng------



,question_id,question_topic,question_content,response_content,question_language
0,46918495,maize,Theophilus asks: What s the easiest way to prevent squirrels from digging out young maize seedlings? Reply Q34 followed by your response.,Q.981 mix some maize with seedling,eng
1,23520242,chicken,Q how long does a layers take from one day to producing the first egg.,Q159 btn 3 to 4 months,eng
2,12589015,cattle,Q which medicine can i give a calf of one yr old who as been weak about 3months,Q deworm,eng
3,3913570,tomato,Q:how can mtu know that the bull wants to mate?,Q33wen it is near the cow it starts climbing the cow,eng
4,13306274,maize,Q What makes our maize not to grow tall and are eaten with some dudus?,"Q4 not applying fertilisers or enough fertilisers to the maize, not weeding",eng
5,15305545,wheat,e what is livestock?,Q172 LIVESTOCK IS ALL KINDS OF ANIMALS.,eng
6,15893341,rabbit,Which type of medicine do we use when I want to deworm the rabbit,Q23 phalvacen,eng
7,30823411,potato,which is the best soil to plant shangi potatoes,Q69 USE RED VOLCANIC SOIL FOUND IN THE HIGHLANDS LIKE NYANDARUA COUNTY,eng
8,22130725,potato,Q What disease attack irish potato and wat is name ov dat germ? Tell Guys,"Q32 Same types like enemies of g nuts use same treatment of tomatoes, cabbages, etc.",eng
9,27091716,rabbit,Q: WHAT ARE THE BEST FEEDS FOR RABIT AND WHAT IS THE MARKET PRICE,"Q75\nBest foods for rabbits are the simple garden produce. Cabbages, carrots, kale, spinach, cucumber, sweet potatoes, lettuce, are just to mention a few among the best foods for rabbits on top of others like tomato and potato leaves.\nCheck the prices at ayour local market.",eng
